In [2]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/danieloliveiradebrito/Projetos/fakenews


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Load dataset

In [3]:
import datasets
from sklearn.model_selection import train_test_split

from fakenews.read_data import read_fake_recogna

df = read_fake_recogna().to_pandas()

train_full, test = train_test_split(df,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=df["label"]
                            )

train, val = train_test_split(train_full,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=train_full["label"]
                            )

ds = datasets.DatasetDict()
ds["train"] = datasets.Dataset.from_pandas(train.reset_index(drop=True))
ds["test"] = datasets.Dataset.from_pandas(test.reset_index(drop=True))
ds["validation"] = datasets.Dataset.from_pandas(val.reset_index(drop=True))


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## BERT

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("adalbertojunior/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from fakenews.train import FakeNewsTrainer

fake_recogna_trainer = FakeNewsTrainer(ds)
tokenized_ds = fake_recogna_trainer.tokenize_ds()
trainer = fake_recogna_trainer.get_trainer(tokenized_ds)

/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1905/1905 [00:00<00:00, 2102.12 examples/s]


In [7]:
# trainer.train()

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("models/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model.predict(tokenized_ds["validation"])
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("model_name")
model = AutoModelForSequenceClassification.from_pretrained("model_name")

# Tokenize the input text
input_text = "Your input text goes here"
encoded_input = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")

# Make the prediction
with torch.no_grad():
    logits = model(**encoded_input).logits
    predicted_labels = torch.argmax(logits, dim=1)

# Convert the predicted labels to human-readable form
predicted_labels = predicted_labels.item()
label_mapping = {0: "Label 1", 1: "Label 2", 2: "Label 3"}  # Replace with your label mapping
predicted_label = label_mapping[predicted_labels]

# Print the predicted label
print("Predicted Label:", predicted_label)

AttributeError: 'BertForSequenceClassification' object has no attribute 'predict'

In [29]:
trainer.evaluate(tokenized_ds["test"])

100%|██████████| 298/298 [00:46<00:00,  6.47it/s]


{'eval_loss': 0.16764046251773834,
 'eval_accuracy': 0.9546409071818563,
 'eval_runtime': 47.6348,
 'eval_samples_per_second': 49.984,
 'eval_steps_per_second': 6.256,
 'epoch': 3.0}

In [38]:
import torch

torch.mean(tokenized_ds["test"]["labels"].float())

tensor(0.4998)

In [40]:
sample_test = test.query("label == 0")["text"].sample().values[0]
print(sample_test)

encoding = fake_recogna_trainer.tokenizer(sample_test, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

outputs = trainer.model(**encoding)
logits = outputs.logits
probs = torch.nn.Sigmoid()(logits.squeeze().cpu())
probs.argmax()

o imagem começar o espalhar whatsapp semana dezembro e comparar publicação dia dezembro site bbc antigo outubro cnn mostrar o mulher o vacinar o coronavírus reinar unir margaret keenan


tensor(0)

# LLM

In [41]:
from dotenv import load_dotenv
from openai import OpenAI
import os

_ = load_dotenv()
openai = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [42]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Brazil?"},
    ],
)

response.model_dump()["choices"][0]["message"]["content"]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'The capital of Brazil is Brasília.'

In [43]:
test.shape

(2381, 2)

0.04750000000000004

In [54]:
from statsmodels.stats.power import TTestPower

power = TTestPower()
effect_size_stand = power.solve_power(
    nobs=200,
    effect_size=None, #standardized effect size, mean divided by the standard deviation.
    power=0.8,
    alpha=0.05,
    alternative="two-sided",
)
effect_size_stand

0.19906466118162786

## Qual o tamanho necessário para avaliar?

H_0: LLM e BERT com mesmo desempenho ($acc = 0.95$)

H_a: LLM e BERT com desempenhos distintos

----

Para MDE = 5%, $\alpha$ = 0.05, tem-se n = 